Настроить TensorFlow.
Импортируйте TensorFlow в свою программу, чтобы начать:

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2024-01-03 15:08:01.819363: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 15:08:01.857754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 15:08:01.858619: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 15:08:02.455971: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0


In [3]:
# Вывести информацию о доступных GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("Доступные GPU:", len(physical_devices))

Доступные GPU: 0


2024-01-03 14:48:54.430156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 14:48:54.439090: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Загрузите набор данных.
Загрузите и подготовьте набор данных MNIST. Преобразуйте данные выборки из целых чисел в числа с плавающей запятой:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Построить модель машинного обучения
Создайте модель tf.keras.Sequential путем наложения слоев.

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

Для каждого примера модель возвращает вектор оценок логитов или логарифмических шансов , по одному для каждого класса.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.32831365,  0.05636165, -0.0896057 , -0.27512854,  0.26071537,
        -0.16117002,  0.00883514,  0.44022313, -0.17805775, -0.36032468]],
      dtype=float32)

Функция tf.nn.softmax преобразует эти логиты в вероятности для каждого класса:

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.13396405, 0.10206599, 0.088204  , 0.07326839, 0.12520762,
        0.0821123 , 0.09732862, 0.14982696, 0.08073726, 0.06728473]],
      dtype=float32)

Примечание. Можно встроить функцию tf.nn.softmax в функцию активации для последнего уровня сети. Хотя это может сделать выходные данные модели более интерпретируемыми, этот подход не рекомендуется, поскольку невозможно обеспечить точный и численно стабильный расчет потерь для всех моделей при использовании выходных данных softmax.

Определите функцию потерь для обучения с использованием losses.SparseCategoricalCrossentropy , которая принимает вектор логитов и индекс True и возвращает скалярную потерю для каждого примера

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Эта потеря равна отрицательной логарифмической вероятности истинного класса: потеря равна нулю, если модель уверена в правильном классе.

Эта необученная модель дает вероятности, близкие к случайным (1/10 для каждого класса), поэтому начальная потеря должна быть близка к -tf.math.log(1/10) ~= 2.3 .

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.4996674

Перед началом обучения настройте и скомпилируйте модель с помощью Model.compile . Установите класс optimizer на adam , установите loss на функцию loss_fn , которую вы определили ранее, и укажите метрику, которая будет оцениваться для модели, установив параметр metrics на accuracy .

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Обучите и оцените свою модель
Используйте метод Model.fit , чтобы настроить параметры модели и минимизировать потери:

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2024-01-03 14:49:27.597654: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 5s 2ms/step - loss: 0.2956 - accuracy: 0.9142
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1427 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1060 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0857 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0773 - accuracy: 0.9762


In [ ]:
Метод Model.evaluate проверяет производительность моделей, обычно на " Validation-set " или " Test-set ".

In [12]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0749 - accuracy: 0.9766 - 561ms/epoch - 2ms/step


[0.07494451850652695, 0.9765999913215637]

In [13]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.0762814e-08, 2.5328690e-09, 2.6702814e-06, 1.6299461e-04,
        1.3559862e-11, 6.9134094e-09, 2.0614043e-13, 9.9983180e-01,
        5.4876633e-08, 2.3623948e-06],
       [1.1660017e-06, 1.3443214e-03, 9.9844486e-01, 1.9169453e-04,
        5.4058515e-13, 8.5973437e-07, 8.6300339e-08, 1.4036336e-10,
        1.7066997e-05, 1.3497280e-09],
       [2.7251849e-07, 9.9904984e-01, 4.8548285e-05, 7.5782614e-06,
        1.7816739e-05, 1.7432659e-06, 4.0024729e-06, 7.6962408e-04,
        9.9317898e-05, 1.2547166e-06],
       [9.9961805e-01, 6.2612943e-10, 4.5678485e-06, 1.2897854e-06,
        1.5887863e-07, 4.7344570e-06, 3.1770367e-04, 5.1061186e-05,
        4.9007348e-08, 2.4370431e-06],
       [4.0729942e-06, 3.6320322e-10, 4.6330792e-06, 1.2134403e-07,
        9.9683940e-01, 4.0948333e-07, 1.9705558e-06, 5.7789384e-05,
        2.7155493e-06, 3.0888477e-03]], dtype=float32)>